In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv
import os
import json
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch.nn as nn
import torchvision.models as models

In [ ]:
from genericpath import isdir
import os
import json
import re
from sympy import false
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
import sys

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

root_dir = "/content/drive/MyDrive/Selected_Storms_curated"

path_to_append = '/content/drive/MyDrive'

if path_to_append in sys.path:
    print('Path already present')
else:
    sys.path.append(path_to_append)
    print('Path appended to sys')

from Utility_funcs.window_dataloader import StormDataLoader, StormDataset

Mounted at /content/drive
Path appended to sys


In [ ]:
# class StormDataset(Dataset):
#     def __init__(self, root: str, storm: str, sequence_length: int, alldata=False) -> None:
#         self.sequence_length = sequence_length
#         self.prefix = root
#         self.storm_name = storm

#        # 图像预处理转换
#         self.transform = transforms.Compose([
#             transforms.Resize((128, 128)),
#             transforms.CenterCrop((128, 128)),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485], std=[0.229]),
#         ])


#         # 加载数据并预处理
#         self.data, self.preprocessed_images = self._load_data()

#         # 时间缩放器
#         self.time_scaler = MinMaxScaler()
#         self.data['Relative_Time'] = self.time_scaler.fit_transform(self.data[['Relative_Time']])
#         self.data['Time_Diff'] = self.data['Relative_Time'].diff()
#         self.data['Time_Diff'].fillna(0, inplace=True)

#     def _load_data(self):
#         datalist = []
#         preprocessed_images = {}

#         # storm_dir = os.path.join(self.prefix, self.storm_name)
#         # if os.path.isdir(storm_dir):
#         #     image_files = [file for file in os.listdir(storm_dir) if file.endswith((".jpg", ".jpeg"))]
#         #     for im in image_files:
#         storm_dir = os.path.join(self.prefix, self.storm_name)
#         if os.path.isdir(storm_dir):
#             image_files = [file for file in os.listdir(storm_dir) if file.endswith((".jpg", ".jpeg"))]
#             for im in image_files:

#                 split_names = re.split(r"[_.]", im)
#                 split_names = [part for part in split_names if part]
#                 name, num = split_names[0], split_names[1]

#                 # 加载并预处理图像
#                 image_path = os.path.join(storm_dir, im)

#                 image = Image.open(image_path)
#                 preprocessed_images[num] = self.transform(image)


#                 # 读取 JSON 数据
#                 json_data = self._load_json(os.path.join(storm_dir, name + "_" + num))
#                 json_data["Id"] = num
#                 datalist.append(json_data)

#         return pd.DataFrame(datalist), preprocessed_images

#     def _load_json(self, file_prefix):
#         with open(file_prefix + "_label.json", "r") as json_file:
#             label_data = json.load(json_file)
#         with open(file_prefix + "_features.json", "r") as json_file:
#             features_data = json.load(json_file)

#         return {
#             "Storm_Name": features_data["storm_id"],
#             "Wind_Speed": int(label_data["wind_speed"]),
#             "Relative_Time": int(features_data["relative_time"]),
#             "Ocean": int(features_data["ocean"]),
#             "Image_Path": file_prefix + ".jpg",
#         }

#     def __len__(self):
#         return len(self.data) - self.sequence_length

#     # def __getitem__(self, idx):
#         # if idx + self.sequence_length >= len(self.data):
#         #     idx = len(self.data) - self.sequence_length - 1

#         # sequence_data = self.data.iloc[idx: idx + self.sequence_length]
#         # images = [self.preprocessed_images[str(id)] for id in sequence_data["Id"]]
#         # out_image = self.preprocessed_images[str(idx+10)]
#         # sample_input = {
#         #     "image": torch.stack(images),
#         #     "outimage": out_image,
#         #     "relative_time": torch.tensor(sequence_data["Relative_Time"].to_numpy(), dtype=torch.float32),
#         #     "time_diff": torch.tensor(sequence_data["Time_Diff"].to_numpy(), dtype=torch.float32),
#         #     "wind_speed": torch.tensor(sequence_data["Wind_Speed"].to_numpy(), dtype=torch.float32),
#         # }

#         # return sample_input
#     def __getitem__(self, idx):
#         # 检查索引是否有效
#         if idx + self.sequence_length + 1 > len(self.data):
#             return {
#                 "image": torch.tensor([]),
#                 "outimage": torch.tensor([]),
#                 "relative_time": torch.tensor([]),
#                 "time_diff": torch.tensor([]),
#                 "wind_speed": torch.tensor([]),
#             }

#         sequence_data = self.data.iloc[idx: idx + self.sequence_length]
#         images = [self.preprocessed_images[str(id)] for id in sequence_data["Id"]]

#         # 获取输出图像
#         out_image_idx = idx + self.sequence_length
#         if str(out_image_idx) not in self.preprocessed_images:
#             return {
#                 "image": torch.tensor([]),
#                 "outimage": torch.tensor([]),
#                 "relative_time": torch.tensor([]),
#                 "time_diff": torch.tensor([]),
#                 "wind_speed": torch.tensor([]),
#             }
#         out_image = self.preprocessed_images[str(out_image_idx)]

#         sample_input = {
#             "image": torch.stack(images),
#             "outimage": out_image,
#             "relative_time": torch.tensor(sequence_data["Relative_Time"].to_numpy(), dtype=torch.float32),
#             "time_diff": torch.tensor(sequence_data["Time_Diff"].to_numpy(), dtype=torch.float32),
#             "wind_speed": torch.tensor(sequence_data["Wind_Speed"].to_numpy(), dtype=torch.float32),
#         }

#         return sample_input


#     def __str__(self):
#         class_string = self.__class__.__name__ + "\n\tlen : %d" % self.__len__()
#         for key, value in self.__dict__.items():
#             if key not in ["data", "preprocessed_images"]:
#                 class_string += "\n\t" + str(key) + " : " + str(value)
#         return class_string


In [ ]:
from torch.utils.data.dataloader import default_collate

def custom_collate_fn(batch):
    # 过滤掉包含零大小张量的批次
    batch = [b for b in batch if b['image'].nelement() != 0]
    if len(batch) == 0:
        return None
    return default_collate(batch)


In [ ]:
# root_dir = '/Users/tl723/Downloads/imperial/youlanda2/storm/'
storm = 'bkh'
sequence_length = 10
dataset = StormDataset(root=root_dir, storm=storm, sequence_length=sequence_length)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True, collate_fn=custom_collate_fn)


TypeError: StormDataset.__init__() got an unexpected keyword argument 'root'

In [ ]:
# 展示 DataLoader 的基本信息
print("DataLoader Information:")
print("-----------------------")
print(f"Batch Size: {dataloader.batch_size}")
print(f"Dataset Size: {len(dataloader.dataset)}")
print(f"Number of Batches: {len(dataloader)}")  # 迭代次数

# 展示一个批次的数据
for i, batch in enumerate(dataloader):
    if batch['image'].nelement() == 0:  # 跳过无效批次
        continue
    print("\nSample Batch:")
    print("-------------")
    print("Images Shape:", batch['image'].shape)
    print("Out Image Shape:", batch['outimage'].shape)
    print("Relative Time Shape:", batch['relative_time'].shape)
    print("Time Difference Shape:", batch['time_diff'].shape)
    print("Wind Speed Shape:", batch['wind_speed'].shape)
    # 只展示一个批次的数据
    break


In [ ]:
# resnet50
# class STA_LSTM(nn.Module):
#     def __init__(self, in_dim, sequence_length, lstm_in_dim, lstm_hidden_dim, cnn_feature_dim, out_dim, use_gpu=False):
#         super(STA_LSTM, self).__init__()

#         # 参数导入部分
#         self.in_dim = in_dim
#         self.sequence_length = sequence_length
#         self.lstm_in_dim = lstm_in_dim
#         self.lstm_hidden_dim = lstm_hidden_dim
#         self.cnn_feature_dim = cnn_feature_dim
#         self.out_dim = out_dim
#         self.use_gpu = use_gpu

#         # CNN特征提取器
#         self.cnn_extractor = models.resnet50(pretrained=True)
#         for param in self.cnn_extractor.parameters():
#             param.requires_grad = False
#         self.fitlinear = nn.Linear(1000,1024)

#         # 网络结构部分
#         self.batch_norm = nn.BatchNorm1d(in_dim)
#         self.layer_in = nn.Linear(in_dim, in_dim, bias=False)
#         self.S_A = nn.Linear(lstm_in_dim, lstm_in_dim)
#         self.lstmcell = nn.LSTMCell(cnn_feature_dim, lstm_hidden_dim)
#         self.T_A = nn.Linear(sequence_length * lstm_hidden_dim, sequence_length)
#         self.layer_out = nn.Linear(lstm_hidden_dim, out_dim, bias=False)


#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(lstm_hidden_dim, 256, kernel_size=4, stride=2, padding=1),  # 256x2x2
#             nn.ReLU(),
#             nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 128x4x4
#             nn.ReLU(),
#             nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 64x8x8
#             nn.ReLU(),
#             nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # 32x16x16
#             nn.ReLU(),
#             nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),  # 16x32x32
#             nn.ReLU(),
#             nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1),  # 8x64x64
#             nn.ReLU(),
#             nn.ConvTranspose2d(8, 4, kernel_size=4, stride=2, padding=1),  # 4x128x128
#             nn.ReLU(),
#             nn.ConvTranspose2d(4, 2, kernel_size=4, stride=2, padding=1),  # 2x256x256
#             nn.ReLU(),
#             nn.ConvTranspose2d(2, 3, kernel_size=4, stride=2, padding=1),  # 3x512x512
#             nn.ReLU(),
#             nn.ConvTranspose2d(3, 3, kernel_size=2, stride=1, padding=0),  # 3x511x511
#             nn.ReLU(),
#             # 裁剪层以精确匹配目标尺寸
#             nn.Conv2d(3, 3, kernel_size=1, stride=1, padding=0)  # 调整为最终尺寸
#         )

#         # 添加一个裁剪操作
#         self.crop = lambda x: x[:, :, :366, :366]

#         # 激活函数
#         self.sigmoid = nn.Sigmoid()
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, input_images):
#         batch_size, sequence_length, C, H, W = input_images.size()
#         cnn_input = input_images.view(batch_size * sequence_length, C, H, W)
#         features = self.cnn_extractor(cnn_input)
#         features = self.fitlinear(features)
#         features = features.view(batch_size, sequence_length, -1)

#         # 初始化隐藏状态和记忆单元
#         h_t_1 = torch.zeros(batch_size, self.lstm_hidden_dim)
#         c_t_1 = torch.zeros(batch_size, self.lstm_hidden_dim)
#         if self.use_gpu:
#             h_t_1 = h_t_1.cuda()
#             c_t_1 = c_t_1.cuda()

#         h_list = []
#         for i in range(sequence_length):
#             x_t = features[:, i, :]
#             alpha_t = self.sigmoid(self.S_A(x_t))
#             alpha_t = self.softmax(alpha_t)

#             h_t, c_t = self.lstmcell(x_t * alpha_t, (h_t_1, c_t_1))
#             h_list.append(h_t)

#             h_t_1, c_t_1 = h_t, c_t

#         total_ht = torch.cat(h_list, dim=1)
#         beta_t = self.relu(self.T_A(total_ht))
#         beta_t = self.softmax(beta_t)

#         out = torch.zeros(batch_size, self.lstm_hidden_dim).cuda() if self.use_gpu else torch.zeros(batch_size, self.lstm_hidden_dim)
#         for i in range(len(h_list)):
#             out = out + h_list[i] * beta_t[:, i].unsqueeze(1)

#         out = out.view(batch_size, self.lstm_hidden_dim, 1, 1)
#         # print('outsize', out.shape())
#         out = self.decoder(out)
#         out = self.crop(out)
#         return out



In [ ]:
#resnet18
class STA_LSTM(nn.Module):
    def __init__(self, in_dim, sequence_length, lstm_in_dim, lstm_hidden_dim, cnn_feature_dim, out_dim, use_gpu=False):
        super(STA_LSTM, self).__init__()

        # 参数导入部分
        self.in_dim = in_dim
        self.sequence_length = sequence_length
        self.lstm_in_dim = lstm_in_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.cnn_feature_dim = cnn_feature_dim
        self.out_dim = out_dim
        self.use_gpu = use_gpu

        # CNN特征提取器
        self.cnn_extractor = models.resnet18(pretrained=True)
        self.cnn_extractor.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 修改第一个卷积层以接受1个通道
        for param in self.cnn_extractor.parameters():
            param.requires_grad = False
        # 调整fitlinear层以匹配resnet18的输出维度（resnet18的全连接层输出维度为512）
        self.fitlinear = nn.Linear(1000, 1024)

        # 网络结构部分
        self.batch_norm = nn.BatchNorm1d(in_dim)
        self.layer_in = nn.Linear(in_dim, in_dim, bias=False)
        self.S_A = nn.Linear(lstm_in_dim, lstm_in_dim)
        self.lstmcell = nn.LSTMCell(cnn_feature_dim, lstm_hidden_dim)
        self.T_A = nn.Linear(sequence_length * lstm_hidden_dim, sequence_length)
        self.layer_out = nn.Linear(lstm_hidden_dim, out_dim, bias=False)


        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=4, stride=2, padding=1),  # 第1层
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 第2层
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 第3层
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # 第4层
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),  # 第5层
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1),  # 第6层
            nn.ReLU(),
            nn.ConvTranspose2d(8, 4, kernel_size=4, stride=2, padding=1),  # 第7层
            nn.ReLU(),
            nn.ConvTranspose2d(4, 2, kernel_size=4, stride=2, padding=1),  # 第8层
            nn.ReLU(),
            nn.ConvTranspose2d(2, 1, kernel_size=4, stride=2, padding=1),  # 第9层，输出变为单通道
            nn.ReLU(),
            nn.Conv2d(1, 1, kernel_size=1, stride=1, padding=0)  # 最终尺寸调整层，确保输出尺寸为1x128x128
        )


        # 更新裁剪操作以适配128x128
        self.crop = lambda x: x[:, :, :128, :128]

        # 激活函数
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_images):
        batch_size, sequence_length, C, H, W = input_images.size()
        cnn_input = input_images.view(batch_size * sequence_length, C, H, W)
        features = self.cnn_extractor(cnn_input)
        features = self.fitlinear(features)
        features = features.view(batch_size, sequence_length, -1)

        # 初始化隐藏状态和记忆单元
        h_t_1 = torch.zeros(batch_size, self.lstm_hidden_dim)
        c_t_1 = torch.zeros(batch_size, self.lstm_hidden_dim)
        if self.use_gpu:
            h_t_1 = h_t_1.cuda()
            c_t_1 = c_t_1.cuda()

        h_list = []
        for i in range(sequence_length):
            x_t = features[:, i, :]
            alpha_t = self.sigmoid(self.S_A(x_t))
            alpha_t = self.softmax(alpha_t)

            h_t, c_t = self.lstmcell(x_t * alpha_t, (h_t_1, c_t_1))
            h_list.append(h_t)

            h_t_1, c_t_1 = h_t, c_t

        total_ht = torch.cat(h_list, dim=1)
        beta_t = self.relu(self.T_A(total_ht))
        beta_t = self.softmax(beta_t)

        out = torch.zeros(batch_size, self.lstm_hidden_dim).cuda() if self.use_gpu else torch.zeros(batch_size, self.lstm_hidden_dim)
        for i in range(len(h_list)):
            out = out + h_list[i] * beta_t[:, i].unsqueeze(1)

        out = out.view(batch_size, self.lstm_hidden_dim, 1, 1)
        # print('outsize', out.shape())
        out = self.decoder(out)
        out = self.crop(out)
        return out



In [ ]:
import torch.optim as optim

# 定义损失函数和优化器
model = STA_LSTM(in_dim=1024, sequence_length=10, lstm_in_dim=1024, lstm_hidden_dim=256, cnn_feature_dim=1024, out_dim=1, use_gpu=False)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 训练循环
num_epochs = 10  # 根据需要调整
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        if batch['image'].nelement() == 0:  # 跳过无效批次
            continue
        inputs = batch['image']
        targets = batch['outimage']
        if model.use_gpu:
            inputs, targets = inputs.cuda(), targets.cuda()
        # 前向传播
        outputs = model(inputs)
        # print(inputs.shape)
        # print(targets.shape)
        # print(outputs.shape)
        loss = criterion(outputs, targets)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # print('*',running_loss / len(dataloader))
    epoch_loss = running_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# 保存模型
torch.save(model.state_dict(), 'model.pth')


In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

def show_image(image_tensor):
    """
    显示模型预测的图像。
    参数:
        image_tensor (torch.Tensor): 预测的图像张量，形状为 [1, channels, height, width].
    """
    # 检查图像张量的形状
    # if image_tensor.ndim != 4 or image_tensor.shape[0] != 1:
    #     raise ValueError("图像张量应该有一个形状为 [1, channels, height, width]")

    # 将图像张量转换回PIL图像以进行显示
    transform = transforms.ToPILImage()
    image = transform(image_tensor.squeeze(0))  # 移除批量维度

    # 显示图像
    plt.imshow(image, cmap='gray')  # 如果是RGB图像，移除 cmap 参数
    plt.axis('off')  # 不显示坐标轴
    plt.show()

# 假设 prediction 是你的预测图像张量
# show_image(prediction)

In [ ]:
def evaluate_model(model, dataloader, device='cpu'):
    """
    评估模型并生成图像。
    参数:
        model: 训练好的模型。
        dataloader: 用于评估的数据加载器。
        device: 运行模型的设备（'cpu' 或 'cuda'）。
    """
    model.eval()  # 将模型置于评估模式
    model.to(device)

    with torch.no_grad():  # 禁用梯度计算
        for i, batch in enumerate(dataloader):
            inputs = batch['image'].to(device)
            targets = batch['outimage'].to(device)
            print(targets.shape)
            # 前向传播，生成图像
            outputs = model(inputs)

            # 可选：显示或保存图像
            # 假设我们只显示/保存第一个图像
            predicted_image = outputs[0].cpu()  # 将图像移回CPU
            print(predicted_image)
            show_image(predicted_image)  # 使用之前定义的显示函数
            show_image(targets[0])
            # 如果只想处理一个批次，可以在这里中断循环
            break

# 加载训练好的模型
model = STA_LSTM(in_dim=1024, sequence_length=10, lstm_in_dim=1024, lstm_hidden_dim=256, cnn_feature_dim=1024, out_dim=1, use_gpu=False)
model.load_state_dict(torch.load('model.pth'))

# 评估模型
evaluate_model(model, dataloader)
